In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import requests
import math
from datetime import datetime, timedelta
from pandas.plotting import register_matplotlib_converters
from htmldate import find_date
from util.api import get_lts_list
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import re
%matplotlib inline
plt.style.use('classic')

In [2]:
lts_list = get_lts_list()
df_list = []
df_by_category_list = []

for idx, lts in enumerate(lts_list):
    df = pd.read_pickle(f'../../data/dfs/lts-{lts}/lts-{lts}.df')
    df_list.append(df)

In [6]:
#Este fragmento de codigo procesa el total de descargas de los paquetes presentes en la ultima lts que tenga el proyecto
total_downloads = 0
index = 0
print('----------Inicio calculo de descargas------------')
for idx,row in df_list[-1].iterrows():
    # Make a request
    url = "https://hackage.haskell.org/package/%s" % row["package"]
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    downloads = 0
    # Extract head of page
    try:
        downloads = 0
        if index == math.trunc(len(df_list[-1])/4):
            print('----------1/4 del progreso------------')
        elif index == math.trunc(len(df_list[-1])/2):
            print('----------2/4 del progreso------------')
        elif index == math.trunc(3*(len(df_list[-1])/4)):
            print('----------3/4 del progreso------------')
        elif index == math.trunc(len(df_list[-1])-1):
            print('----------4/4 del progreso------------')
        datas = soup.find(id = 'properties')
        downloads = datas.find("th", string="Downloads").parent.select("td")
        downloads = str(downloads[0]).replace("<td>", "").split(" ",1)
        total_downloads += int(downloads[0])
    except:
        print(row["package"])
        print('----------------------')
    index += 1
    # print the result

print("Total de descargas en la lts",total_downloads)

----------Inicio calculo de descargas------------
Total de descargas en la lts 85957416


In [3]:
#Esto se encarga de obtener la descripcion de los paquetes presente en stackage

url = "https://www.stackage.org/lts-20.15"  # Si se desea obtener la data de otra lts se modifica la url (19.11)
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
# Extract head of page
package_description = []

datas = soup.find('tbody')
columnas = datas.find_all("td")
for idx,dato in enumerate(columnas):
    if idx%2:
        package = str(columnas[idx-1]).split('</a')[0].split('">')[1]
        result = re.search(r"(.*)-.*$", package).group(1)
        description = str(dato).split('</')[0].split('td>')[1]
        package_description.append([result,description])

In [4]:
columns = ["Paquetes","Descripcion"]
columns.extend(lts_list)

In [5]:
pkg_with_monad_descr = []
pkg_with_all = []
for pkg,descr in package_description:
    descrLower = descr.lower()
    if "monad" in descrLower:
        pkg_with_monad_descr.append(pkg)
        pkg_with_all.append([pkg,descr])
pkg_with_monads = [[] for i in range(len(lts_list))]
totalPkg = ["Total paquetes","Total de paquetes por LTS"]
for idx,df in enumerate(df_list):
    print('|------------------------------|')
    print(lts_list[idx])
    print("cantidad de veces importado equivalente al 0,1%:",len(df)*0.01)
    totalPkg.append(len(df))
    indiceMonads = 0
    for monad in pkg_with_monad_descr:
        usage = 0
        for jdx,row in df.iterrows():
            if monad  in row['deps']:
                usage += 1
        
        if not df[df['package']==monad].empty:
            pkg_with_all[indiceMonads].append(usage)
            pkg_with_monads[idx].append(df[df['package']==monad]['package'])
            if usage >= len(df)*0.01:
                print(monad,' es importado un total  de: ',usage)
        else:
            pkg_with_all[indiceMonads].append(-1)
        indiceMonads+=1
pkg_with_all.append(totalPkg)


|------------------------------|
0-7
cantidad de veces importado equivalente al 0,1%: 8.27
comonad  es importado un total  de:  21
mmorph  es importado un total  de:  9
monad-control  es importado un total  de:  54
monad-logger  es importado un total  de:  14
mtl  es importado un total  de:  179
parsec  es importado un total  de:  45
transformers  es importado un total  de:  209
|------------------------------|
2-22
cantidad de veces importado equivalente al 0,1%: 10.66
comonad  es importado un total  de:  23
free  es importado un total  de:  13
mmorph  es importado un total  de:  13
monad-control  es importado un total  de:  61
monad-logger  es importado un total  de:  19
mtl  es importado un total  de:  238
parsec  es importado un total  de:  52
transformers  es importado un total  de:  288
|------------------------------|
3-22
cantidad de veces importado equivalente al 0,1%: 13.52
comonad  es importado un total  de:  25
free  es importado un total  de:  18
mmorph  es importado un to

In [6]:
df_excel = pd.DataFrame(pkg_with_all,columns = columns)
df_excel

,Paquetes,Descripcion,0-7,2-22,3-22,6-35,7-24,9-21,11-22,12-14,...,15-3,16-11,16-31,17-2,18-6,18-8,18-28,19-33,20-11,20-15
0,abstract-par,Type classes generalizing the functionality of...,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
1,asn1-parse,Simple monadic parser for ASN1 stream types,1,1,2,3,2,2,2,2,...,2,2,2,2,2,2,2,1,1,1
2,attoparsec-base64,"Fetch only base64 characters, erroring in the ...",-1,-1,-1,-1,-1,-1,-1,0,...,0,0,0,0,0,0,0,0,0,0
3,backtracking,A backtracking monad,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,1,1,1,1,0,0,0
4,boots,IoC Monad in Haskell,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,xmonad-extras,Third party extensions for xmonad with wacky d...,-1,-1,-1,-1,-1,0,0,-1,...,0,0,0,0,0,0,0,0,0,0
131,yjtools,"some tools for Monad, List, Tuple and so on",-1,-1,-1,0,0,2,2,0,...,2,2,2,1,1,1,1,0,0,0
132,zio,App-centric Monad-transformer based on Scala Z...,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,0,0,0,0,0,0,0,0
133,zipper-extra,Zipper utils that weren't in Control.Comonad.S...,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,1,1,1,1,0,1,1


In [7]:
df_excel.to_excel('package_with_monads.xlsx', sheet_name='Total_pkg_monad', index=False)